In [8]:
import json
import spacy # pré-traitement
from unidecode import unidecode # Suppression des accents
import re

In [9]:
#python -m spacy download fr_core_news_md

Réception des données

In [10]:
with open('../data/req_makeorg_environnement.json', encoding = "utf8") as f:
    data = json.load(f)['results']

Chargement du modèle linguistique (tokénisation, lemmes, stop words...)

In [11]:
def initialisation_nlp() -> spacy.Language:
    nlp = spacy.load("fr_core_news_md")
    stop_words_supp = {'>', '|', '<', '->', 'etc', 'etc.', '', 'ca', '=', 'e',}
    nlp.Defaults.stop_words |= stop_words_supp
    return nlp

In [12]:
nlp = initialisation_nlp()

Application du modèle et récupération des lemmes pour chaque proposition

In [17]:
def pre_traitement_documents(documents: str | list[str], nlp: spacy.Language = None) -> list[list[str]]:
    if nlp is None:
        nlp = initialisation_nlp()

    if isinstance(documents, str):
        documents = [documents]

    docs = []

    for doc in documents:
        # Récupération de la phrase, suppression des accents
        phrase = unidecode(doc['content'])

        # passage en minuscules
        phrase = phrase.lower()

        # Remplacement (ou suppression) de certains caractères
            # Chiffres
        phrase = phrase.replace("co2", "co_deux")
        phrase = "".join(i for i in phrase if not i.isdigit())
        phrase.replace("co_deux", "co2")

            # Symboles
        phrase = phrase.replace('|', ' ')
        phrase = phrase.replace('&', 'et')
        phrase = phrase.replace('->', ' ')
        phrase = phrase.replace('(s)','')
        phrase = phrase.replace('.e.s', '')
        phrase = phrase.replace('(', '')
        phrase = phrase.replace(')', '')
        phrase = phrase.replace('-', ' ')
        phrase = phrase.replace('/', ' ')
        phrase = phrase.replace('.', ' ')
            # Erreurs
        phrase = phrase.replace('toogoodtogo', '')
        phrase = phrase.replace('passnavigo', 'pass navigo')
            # Unions
        unions = {"mc do": 'macdonald', "fast food": 'fastfood', "costa rica": 'costarica', "union europeenne": 'unioneuropeenne'}

        for u in unions:
            phrase = phrase.replace(u, unions[u])

        # Expressions régulières
        expreg: list[list[str]] = []
        expreg.append(["transport.{0,5}communs?", "transport_commun"])
        expreg.append(["voyage.{0,5}communs?", "transport_commun"])
        expreg.append(["politiques?\spubliques?", "politique_publique"])
        expreg.append(["huile.{0,5}palmes?","huile_de_palm"])
            # Suppression des s en fin de mots
        #expreg.append(["s\s+", ' ']) 
        #expreg.append(["s$", ''])
        for exp in expreg:
            phrase = reg = re.compile(exp[0]).sub(exp[1], phrase)
            

            
        # Tokenisation
        phrase = nlp(phrase)

        # Suppression de "il faut" en début de phrase
        phrase = phrase[2:]

        # Ajout de quelques conversions à faire au niveau du lemme
        conversion = {'km': 'kilometre', 'cm': 'centimetre', 'in': 'dans', 'made': 'fabrique', 'ue': 'unioneuropeenne', 'wc': "toilette"}

        docs.append([])
        for token in phrase:
            
            lemme = token.lemma_.replace(' ', '')

            # Suppression ponctuation et stop-word
            # On ne garde que les lemmes de longueurs > 1 (enlève les lettres libres et les cases vides)
            if not token.is_stop and not token.is_punct and len(lemme) > 1:
                
                if lemme in conversion:
                    lemme = conversion[lemme]

                docs[-1].append(lemme)

    return docs[0] if len(docs) == 1 else docs

In [18]:
docs = pre_traitement_documents(data, nlp)

Suppression des redondances

In [9]:
if False:
    print(f"Nombre de documents avant suppression des doublons : {len(docs)}")

    docsSansDoublons = []
    [docsSansDoublons.append(doc) for doc in docs if doc not in docsSansDoublons]

    print(f"Nombre de documents après suppression des doublons : {len(docsSansDoublons)}")
    docs = docsSansDoublons

Enregistrement des données

In [16]:
with open("../data/docs.json", 'w') as fichierJson:
    json.dump(docs, fichierJson)